# Statistical Learning and Deep Learning HW3



### Q1

#### Q1.1

1. Read the dataset.

In [138]:
import numpy as np
import pandas as pd
import pickle
from sklearn import preprocessing

df = pd.read_csv('./ds/namesex_data_v2.csv', sep=',')
print(df.head())

  gname  sex  fold
0    承憲    1     9
1    均平    1     7
2    思安    0     6
3    佑誠    1     3
4    乃馨    0     0


2. Split the dataset into training, validation, stack, and test.

In [139]:
# split by fold value
train = df[df['fold'] <= 6]
valid = df[df['fold'] == 7]
stack = df[df['fold'] == 8]
test = df[df['fold'] == 9]
    
train = train.reset_index()
valid = valid.reset_index()
stack = stack.reset_index()
test = test.reset_index()

# split x, y
x_train = train['gname']
y_train = train['sex'].to_numpy()
x_valid = valid['gname']
y_valid = valid['sex'].to_numpy()
x_stack = stack['gname']
y_stack = stack['sex'].to_numpy()
x_test = test['gname']
y_test = test['sex'].to_numpy()

print(f'x_train: {x_train.shape}, y_train: {y_train.shape}')
print(f'x_valid: {x_valid.shape}, y_valid: {y_valid.shape}')
print(f'x_stack: {x_test.shape}, y_stack: {y_stack.shape}')
print(f'x_train: {x_train.shape}, y_train: {y_train.shape}')

x_train: (7483,), y_train: (7483,)
x_valid: (1110,), y_valid: (1110,)
x_stack: (1064,), y_stack: (1073,)
x_train: (7483,), y_train: (7483,)


3. Apply one-hot encoding.

In [156]:
feature = list()
for name in list(x_train.to_numpy()):
    if len(name) > 1:
        feature += list(name)
    feature.append(name)
feature_set = set(feature) - set(' ')
feature_count = dict((f, feature.count(f)) for f in feature_set)
feature_count = dict(filter(lambda pair: pair[1] >= 2, feature_count.items()))
feature = list(feature_count.keys())
print(len(feature))

1629
